In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
import pandas as pd

In [3]:
import dataframe_image as dfi

In [4]:
from plot_config import *
from utils import *

In [5]:
rerun = False

In [6]:
if rerun:
    final = []
    
    for case in ['fresh','salty']:
        print(case)
        map,table = open_case(case)
        
        for duration in [1,2,5,10]:
            print(duration)
            this_sum = 0
            for var in rhs_list:
                print(var, end = ' ')
                contr = float(table[var].isel(time = slice(0,int(duration*365))).sum(dim = 'time').mean(dim = 'space'))
                final.append([case,duration,var,contr])
                this_sum+= contr
            print()
    
            upper = np.minimum(len(table.time)-1,int(duration*365))
            init = float(table['sf'][upper].mean(dim = 'space'))
            endd = float(table['sl'][0].mean(dim = 'space'))
            final.append([case,duration,'init',init])
            final.append([case,duration,'Error',endd-init+this_sum])
    
    final = np.array(final)
    
    np.save('lineplotprep/table1.npy',final)

In [7]:
final = np.load('lineplotprep/table1.npy')
# final

In [8]:
df = pd.DataFrame(final, columns = ['Event','Duration (yr)','Term','Contribution'])
df['Contribution'] = df['Contribution'].astype(float)
df['Contribution'][df['Term'] == 'init'] = -df['Contribution'][df['Term'] == 'init']
df['Contribution'][df['Term'] == 'Error'] = -df['Contribution'][df['Term'] == 'Error']
df

/tmp/ipykernel_811915/3011890810.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Contribution'][df['Term'] == 'init'] = -df['Contribution'][df['Term'] == 'init']
/tmp/ipykernel_811915/3011890810.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Contribution'][df['Term'] == 'Error'] = -df['Contribution'][df['Term'] == 'Error']


,Event,Duration (yr),Term,Contribution
0,fresh,1,e_ua,-0.000336
1,fresh,1,E,0.029679
2,fresh,1,dif_h,-0.032616
3,fresh,1,dif_v,-0.072322
4,fresh,1,A,0.082063
...,...,...,...,...
67,salty,10,A,-0.096720
68,salty,10,I,-0.012248
69,salty,10,F,-0.035441
70,salty,10,init,-0.006083


In [9]:
event_dic = {
    'fresh':'June, 2016',
    'salty':'Jan., 2007'
}
t_dic = {
    'init': 'Initial Anomaly',
    'Error':'Error',
    'Total':'Total'
}
t_dic.update(case_term_dic)
df['Event'] = [event_dic[i] for i in df['Event']]
# df['Contribution'] = [((-1)**int('nit' not in df['Term'][j]))*float(i) for j,i in enumerate(df['Contribution'])]
df['Contribution'] = [-float(i) for j,i in enumerate(df['Contribution'])]
df['Term'] = [t_dic[i] for i in df['Term']]

In [10]:
correct_order = [t_dic[i] for i in rhs_list+['init','Error','Total']]

In [11]:
df = df.set_index('Term').pivot(columns = ['Event', 'Duration (yr)'],values='Contribution')

In [12]:
total = np.zeros_like(df.loc['Error'])
for i in rhs_list+['init','Error']:
    total+=np.array(df.loc[t_dic[i]])
    # print(df.loc[t_dic[i]])

In [13]:
df.loc['Total'] = total
df = df.reindex(correct_order)

In [14]:
df.loc['Total']

Event       Duration (yr)
June, 2016  1               -0.154584
            2               -0.154584
            5               -0.154584
            10              -0.154584
Jan., 2007  1                0.150311
            2                0.150311
            5                0.150311
            10               0.150311
Name: Total, dtype: float64

In [15]:
vmax = 0.2
def to2f(num):
    return f'{num:.3f}'
def make_pretty(styler):
    styler.format(to2f)
    styler.background_gradient(axis=None, vmin=-vmax, vmax=vmax, cmap=balance)
    return styler
styled = df.style.pipe(make_pretty)
styled

In [16]:
dfi.export(styled, 'Table1.png')

OSError: Chrome executable not able to be found on your machine